In [1]:
import mdtraj as mdt
import nglview as nv

from matplotlib.colors import to_rgb
from ipywidgets import interact, fixed, IntSlider

import openpharmacophore.data as data

In [2]:
traj = mdt.load(data.trajectories["eralpha_small.h5"])
traj

/home/daniel/miniconda3/envs/molsys/lib/python3.7/site-packages/mdtraj/core/trajectory.py:438: UserWarning: top= kwargs ignored since this file parser does not support it
  warnings.warn('top= kwargs ignored since this file parser does not support it')


<mdtraj.Trajectory with 3 frames, 27425 atoms, 8028 residues, and unitcells at 0x7f18d3d88c50>

In [3]:
n_frames = traj.n_frames

In [4]:
centers = [
    [34.50, 15.36, 24.73],
    [32.68, 18.35, 24.05],
    [27.88, 23.09, 24.89],
]
color = list(to_rgb("#3498DB"))

## Visualizing

In [5]:
def add_sphere(view, center, color,
               name, comp_num):
    """ Add a sphere to a view 
        
        Parameters
        ----------
        view : nglview.NGLWidget
        center : list[float]
        color : list[float]
        name : str
        comp_num : 1
    """
    js_code = f'this.addShape("{name}", [["sphere", {center}, {color}, 1]])'
    view._execute_js_code(js_code)
    view.update_representation(component=comp_num, repr_index=0, opacity=0.5)

In [6]:
def remove_sphere(view, name):
    """ Remove a sphere from the view.
    
        Parameters
        ----------
        view : nglview.NGLWidget
        name : str
    """
    view._execute_js_code(
    f'this.stage.removeComponent(this.stage.getComponentsByName("{name}").first)'
    )

In [7]:
view = nv.NGLWidget()
view.add_component(traj[1])


add_sphere(view, centers[1], color, "BlueSphere", 1)
view

NGLWidget()

In [8]:
remove_sphere(view, "BlueSphere")

In [9]:
view._ngl_component_names

['nglview.adaptor.MDTrajTrajectory']

In [10]:
view.picked

{}

## Interactive visualization

In [11]:
def show_traj(view, frame):
    """ Show a trajectory with a sphere interactively. 
    """
    global comp, traj
    name = "BlueSphere"
    
    remove_sphere(view, name)
    if comp is not None:
        view.remove_component(comp)

    traj_slice = traj[frame]
    # view.clear()
    comp = view.add_component(traj_slice)
    
    n_components = len(view._ngl_component_ids)
    add_sphere(view, centers[frame], color, name, n_components)

In [12]:
view = nv.NGLWidget()
comp = None

slider = IntSlider(min=0, max=n_frames-1, step=1, value=0)
interact(show_traj, view=fixed(view), frame=slider)
view

interactive(children=(IntSlider(value=0, description='frame', max=2), Output()), _dom_classes=('widget-interac…

NGLWidget()